In [1]:
import os 
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, Xception, VGG19
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten, Reshape, Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, concatenate, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.callbacks  import ModelCheckpoint
from moviepy.editor import * 
from natsort import natsorted
from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
#@title Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title Local
os.chdir("E:/")

In [4]:
## раскадровка фильмов для сбора базы. Ссылка на собранную базу https://drive.google.com/file/d/1MQ36P2OpQDoO62_nqkk7uuuKlPIcdCd5/view?usp=sharing

count = 0                         
imId = 'Мистер Судьба - 720HD - [ KinoMobi.net ].mp4'
cap= cv2.VideoCapture(imId)
frameRate = cap.get(5)*3
image_folder = 'forsort'
while cap.isOpened():
  frameId = cap.get(1)
  ret, frame = cap.read()
  if (ret != True):
    break
  if (frameId % np.floor(frameRate) == 0):
      filename ="frameF%d.jpg" % count;count+=1
      cv2.imwrite(image_folder+'/'+ filename, frame)
cap.release()
cv2.destroyAllWindows()

In [ ]:
!unzip -q '/content/drive/MyDrive/Базы/Jim.zip'

## vgg16

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split= 0.2, horizontal_flip=True, rotation_range=5)

train_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(224,224), subset='training', batch_size=64)
val_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(224,224), subset='validation', batch_size=64)

Found 9684 images belonging to 2 classes.
Found 2420 images belonging to 2 classes.


In [11]:
vgg_model = VGG16(include_top=False)
vggModel = Model(vgg_model.input,vgg_model.output)

xc_model = Xception(include_top=False)
xcModel = Model(xc_model.input,xc_model.output)

inputs = Input(shape=(224,224,3))
popOut = vggModel(inputs)
flatten = Flatten()(popOut)
dense1 = Dense(1024, activation = 'relu')(flatten)
drop1 = Dropout(0.5)(dense1)
dense2 = Dense(1024, activation = 'relu')(drop1)
drop2 = Dropout(0.5)(dense2)
outputs = Dense(2, activation = 'softmax')(drop2)
model = Model(inputs, outputs)
vggModel.trainable = False

model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])
filepath="best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit(train_generator,  epochs=7, validation_data=val_generator, callbacks = callbacks_list)

In [ ]:
model.load_weights('best_model.h5')
model.save('E:\content\drive\MyDrive\TrainedModels\jimVGGVal78')

INFO:tensorflow:Assets written to: E:\content\drive\MyDrive\TrainedModels\jimVal78\assets


## Xception

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split= 0.2, horizontal_flip=True, rotation_range=5)

train_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(229,229), subset='training', batch_size=64)
val_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(229,229), subset='validation', batch_size=64)

Found 9684 images belonging to 2 classes.
Found 2420 images belonging to 2 classes.


In [7]:
xc_model = Xception(include_top=False)
xcModel = Model(xc_model.input,xc_model.output)

inputs = Input(shape=(229,229,3))
popOut = xcModel(inputs)
flatten = Flatten()(popOut)
dense1 = Dense(1024, activation = 'relu')(flatten)
drop1 = Dropout(0.5)(dense1)

outputs = Dense(2, activation = 'softmax')(drop1)
model = Model(inputs, outputs)
xcModel.trainable = False

model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])
filepath="best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [8]:
model.fit(train_generator,  epochs=10, validation_data=val_generator, callbacks = callbacks_list)

Epoch 1/10
152/152 [==============================] - 116s 749ms/step - loss: 2.0713 - accuracy: 0.7783 - val_loss: 0.6416 - val_accuracy: 0.7471

Epoch 00001: val_accuracy improved from -inf to 0.74711, saving model to best_model.h5
Epoch 2/10
152/152 [==============================] - 114s 747ms/step - loss: 0.3005 - accuracy: 0.8753 - val_loss: 0.5800 - val_accuracy: 0.7653

Epoch 00002: val_accuracy improved from 0.74711 to 0.76529, saving model to best_model.h5
Epoch 3/10
152/152 [==============================] - 113s 747ms/step - loss: 0.2602 - accuracy: 0.8921 - val_loss: 0.6535 - val_accuracy: 0.7277

Epoch 00003: val_accuracy did not improve from 0.76529
Epoch 4/10
152/152 [==============================] - 113s 746ms/step - loss: 0.2396 - accuracy: 0.9013 - val_loss: 0.5954 - val_accuracy: 0.7678

Epoch 00004: val_accuracy improved from 0.76529 to 0.76777, saving model to best_model.h5
Epoch 5/10
152/152 [==============================] - 114s 747ms/step - loss: 0.2093 - acc

In [9]:
model.load_weights('best_model.h5')
model.save('E:\content\drive\MyDrive\TrainedModels\jimXcVal77')

INFO:tensorflow:Assets written to: E:\content\drive\MyDrive\TrainedModels\jimXcVal77\assets


## vgg19


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split= 0.2, horizontal_flip=True, rotation_range=5)

train_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(224,224), subset='training', batch_size=64)
val_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(224,224), subset='validation', batch_size=64)

Found 9684 images belonging to 2 classes.
Found 2420 images belonging to 2 classes.


In [8]:
vgg_model = VGG19(include_top=False)
vggModel = Model(vgg_model.input,vgg_model.output)

inputs = Input(shape=(224,224,3))
popOut = vggModel(inputs)
flatten = Flatten()(popOut)
dense1 = Dense(1024, activation = 'relu')(flatten)
drop1 = Dropout(0.5)(dense1)

outputs = Dense(2, activation = 'softmax')(drop1)
model = Model(inputs, outputs)
vggModel.trainable = False

model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])
filepath="best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit(train_generator,  epochs=20, validation_data=val_generator, callbacks = callbacks_list)

Epoch 1/20
152/152 [==============================] - 127s 810ms/step - loss: 0.8997 - accuracy: 0.7120 - val_loss: 0.6216 - val_accuracy: 0.6789

Epoch 00001: val_accuracy improved from -inf to 0.67893, saving model to best_model.h5
Epoch 2/20
152/152 [==============================] - 121s 795ms/step - loss: 0.4104 - accuracy: 0.8105 - val_loss: 0.5651 - val_accuracy: 0.6934

Epoch 00002: val_accuracy improved from 0.67893 to 0.69339, saving model to best_model.h5
Epoch 3/20
152/152 [==============================] - 122s 800ms/step - loss: 0.3830 - accuracy: 0.8256 - val_loss: 0.5882 - val_accuracy: 0.6847

Epoch 00003: val_accuracy did not improve from 0.69339
Epoch 4/20
152/152 [==============================] - 121s 793ms/step - loss: 0.3621 - accuracy: 0.8427 - val_loss: 0.6350 - val_accuracy: 0.6579

Epoch 00004: val_accuracy did not improve from 0.69339
Epoch 5/20
152/152 [==============================] - 120s 792ms/step - loss: 0.3783 - accuracy: 0.8267 - val_loss: 0.6082 - 

In [6]:
model.load_weights('best_model.h5')
model.save('E:\content\drive\MyDrive\TrainedModels\jimvgg19Val77')

INFO:tensorflow:Assets written to: E:\content\drive\MyDrive\TrainedModels\jimvgg19Val77\assets


## обработка

In [ ]:
def take_frames_from_video_by_seconds(model, video_path, divide_sec_on):
  second = 0 
  cap = cv2.VideoCapture(video_path)
  frameRate = cap.get(5)/divide_sec_on
  image_folder = 'temp'
  while cap.isOpened():
    frameId = cap.get(1)
    ret, frame = cap.read()
    if not ret:
      break
    if frameId%round(frameRate,0)==1:  
      filename =f"{round(second, 1)}.jpg";second+=(1/round(frameRate,0))
      cv2.imwrite(image_folder+'/'+ filename, cv2.resize(frame, (224,224)))
  cap.release()
  cv2.destroyAllWindows()
  return round(frameRate,0)
  
def predict_from_frames(frameRate, model=model, path='temp'):
  counter = 0
  predict = []
  temp = []
  for filename in tqdm(natsorted(os.listdir(path))):

      img = cv2.imread(path+'/'+filename)
      img = img/255.
      pred = model.predict(img[None, :,:,:])
      temp.append(np.argmax(pred[0]))
      counter+=1
      if counter>=frameRate:
        predict.append(temp)
        temp = []
        counter = 0
  return np.array(predict)

def get_timings_from_predict(predict, frameRate):
  seconds = []
  for s, pred in enumerate(predict):
    if sum(pred)<frameRate*0.4:
      seconds.append(s)
  return seconds

def preprocess_timings(list_of_seconds, bias): # bias - разница между планами. Не разрезать кусок видео, если между от джима до джима промежуток bias секунд(0 - полная нетерпимость к отсутствию джима)
  time_list = []
  temp = []
  for i in range(len(list_of_seconds)-1):
    temp.append(list_of_seconds[i])
    if list_of_seconds[i+1]-list_of_seconds[i]>bias+1:
      time_list.append([temp[0], temp[-1]+1])
      temp = []
  return time_list

def extract_samples(filepath, time_list):
  for timings in time_list:
    clip = VideoFileClip(filepath) 
    clip = clip.subclip(timings[0], timings[1]) 
    clip.write_videofile(filename = f"cut/{timings[0]}.mp4")
    clear_output()
    clip.close()

def make_one_clip_from_cuts(cutdir="cut/"):
  clips =[]
  for root, dirs, files in os.walk(cutdir):
      files = natsorted(files)
      for file in tqdm(files):
        filePath = os.path.join(root, file)
        video = VideoFileClip(filePath)
        clips.append(video)
  final_clip = concatenate_videoclips(clips)
  final_clip.write_videofile('output.mp4', logger=None, verbose=False)

In [ ]:
def main(filepath,model=model, fps_division = 5, bias = 1):
  try: os.mkdir('temp'); os.mkdir('cut')
  except Exception:
    pass
  print('Подготовка к распознаванию')
  divided_fps = take_frames_from_video_by_seconds(model, filepath, fps_division)
  clear_output()
  print('Распознавание')
  predict = predict_from_frames(divided_fps)
  clear_output()
  print('Расчет длительности клипа')
  seconds = get_timings_from_predict(predict, divided_fps)
  time_list = preprocess_timings(seconds, bias)
  clear_output()
  print('Нарезка клипов')
  extract_samples(filepath, time_list)
  clear_output()
  print('Склеивание клипов. Расчетное время Джима в кадре:',round(len(seconds)/60,0), "минут", len(seconds)%60, 'секунд')
  make_one_clip_from_cuts()

In [ ]:
main('Shou_Trumana_Kinosimka.RU (online-video-cutter.com).mp4')

  1%|▌                                                                                 | 1/162 [00:00<00:32,  4.98it/s]

Склеивание клипов. Расчетное время Джима в кадре: 7.0 минут 0 секунд


100%|████████████████████████████████████████████████████████████████████████████████| 162/162 [00:32<00:00,  5.01it/s]
